# Generate min/max repesentative single-cell images per top two Null features from the model

1. Average edge intensity of GFP in cytoplasm
2. Radial distribution of RFP in cytoplasm

## Import libraries

In [1]:
import pathlib
from pprint import pprint

import cv2
import pandas as pd
from typing import List, Dict

## Define functions

In [2]:
# Function for formatting min/max row data frames into dictionaries
def create_sc_dict(dfs: List[pd.DataFrame], names: List[str]) -> dict:
    """Format lists of data frames and names into a dictionary with all relevant metadata to find single-cell images.

    Args:
        dfs (List[pd.DataFrame]): List of data frames each containing a single cell and relevant metadata.
        names (List[str]): List of names corresponding to the data frames.

    Returns:
        dict: Dictionary containing info relevant for finding single-cell crops.
    """
    sc_dict = {}
    for df, name in zip(dfs, names):
        for i, (_, row) in enumerate(df.iterrows()):
            key = f"{name}_{i + 1}"
            sc_dict[key] = {
                "plate": row["Metadata_Plate"],
                "well": row["Metadata_Well"],
                "site": row["Metadata_Site"],
                "location_center_x": row["Metadata_Nuclei_Location_Center_X"],
                "location_center_y": row["Metadata_Nuclei_Location_Center_Y"],
            }
    return sc_dict

In [3]:
# Function for generating and saving single-cell crops per channel as PNGs
def generate_sc_crops(
    sc_dict: Dict,
    channel_mapping: Dict[int, str],
    images_dir: pathlib.Path,
    output_img_dir: pathlib.Path,
    crop_size: int,
) -> None:
    """Using a dictionary with single-cell metadata info per image set, single-cell crops per channel are generated
    and saved as PNGs in an image set folder.

    Args:
        sc_dict (Dict): Dictionary containing info relevant for finding single-cell crops.
        channel_mapping (Dict[int, str]): Dictionary mapping integer to channel name for generating paths.
        images_dir (pathlib.Path): Directory where illumination corrected images are found.
        output_img_dir (pathlib.Path): Main directory to save each image set single-cell crops
        crop_size (int): Size of the box in pixels (example: setting crop_size as 250 will make a 250x250 pixel crop
        around the single-cell center coordinates)
    """
    for key, info in sc_dict.items():
        # Initialize a list to store file paths for every image set
        file_paths = []

        # Create file paths with well, site, and channel
        for i in range(1, 5):  # Update the range to start from 1
            channel = channel_mapping[i]
            filename = f"{images_dir}/{info['well']}_01_{i}_{info['site']}_{channel}_001_illumcorrect.tiff"
            file_paths.append(filename)

            # Read the image
            channel_image = cv2.imread(filename, cv2.IMREAD_UNCHANGED)

            # Use the location_center_x and location_center_y to create a crop
            center_x = info.get("location_center_x")
            center_y = info.get("location_center_y")

            # Crop dimensions (including crop_size)
            half_crop = crop_size // 2

            # Ensure the center coordinates are valid
            if center_x is not None and center_y is not None:
                # Calculate crop boundaries
                top_left_x = max(int(center_x - half_crop), 0)
                top_left_y = max(int(center_y - half_crop), 0)
                bottom_right_x = min(int(center_x + half_crop), channel_image.shape[1])
                bottom_right_y = min(int(center_y + half_crop), channel_image.shape[0])

                # Perform cropping
                cropped_channel = channel_image[
                    top_left_y:bottom_right_y, top_left_x:bottom_right_x
                ]

                # Ensure the cropped image is of size 250x250
                cropped_channel = cv2.resize(cropped_channel, (crop_size, crop_size))

                # Make directory for the key to keep all channels for an image in one folder
                key_dir = pathlib.Path(f"{output_img_dir}/{key}")
                key_dir.mkdir(exist_ok=True, parents=True)

                # Save the cropped image with single_cell and channel information
                output_filename = pathlib.Path(f"{key_dir}/{key}_d{i}_cropped.png")

                # Check if the file already exists
                if not output_filename.exists():
                    cv2.imwrite(str(output_filename), cropped_channel)
                else:
                    print(f"File {output_filename} already exists. Skipping.")

## Set paths and variables

In [4]:
# Path to cell painting data directory
cell_painting_dir = pathlib.Path(
    "/media/18tbdrive/1.Github_Repositories/nf1_schwann_cell_painting_data"
)

# Images directory for plate 5 (using for finding single-cells)
images_dir = pathlib.Path(
    f"{cell_painting_dir}/1.cellprofiler_ic/Corrected_Images/Corrected_Plate_5"
).resolve(strict=True)

# Output dir for cropped images
output_img_dir = pathlib.Path("./sc_crops")
output_img_dir.mkdir(exist_ok=True)

# Define the size of the cropping box (250x250 pixels)
crop_size = 250

# Define a mapping for the suffixes
channel_mapping = {1: "DAPI", 2: "GFP", 3: "CY5", 4: "RFP"}

# Create open list for one row data frames for each top feature per channel per cell type
list_of_dfs = []

# Create open list of names to assign each data frame in a list relating to the feature, channel, and cell type
list_of_names = []

## Load in Plate 5 data to generate repesentative images from

In [5]:
# Load in normalized + feature selected data as data frame
plate5_df = pd.read_parquet(
    pathlib.Path(
        f"{cell_painting_dir}/3.processing_features/data/single_cell_profiles/Plate_5_sc_feature_selected.parquet"
    )
)

# Load in annotated dataframe to extract neighbors
annot_df = pd.read_parquet(
    pathlib.Path(
        f"{cell_painting_dir}/3.processing_features/data/single_cell_profiles/Plate_5_sc_annotated.parquet"
    ),
    columns=[
        "Metadata_Well",
        "Metadata_Site",
        "Metadata_Nuclei_Number_Object_Number",
        "Cells_Neighbors_NumberOfNeighbors_Adjacent",
    ],
)

plate5_df = plate5_df.merge(
    annot_df,
    on=["Metadata_Well", "Metadata_Site", "Metadata_Nuclei_Number_Object_Number"],
    how="inner",
)

plate5_df.rename(
    columns={
        "Cells_Neighbors_NumberOfNeighbors_Adjacent": "Metadata_Number_of_Cells_Neighbors_Adjacent"
    },
    inplace=True,
)

print(plate5_df.shape)
plate5_df.head()

(5793, 1175)


,Metadata_WellRow,Metadata_WellCol,Metadata_Well,Metadata_Site,Metadata_number_of_singlecells,Metadata_gene_name,Metadata_genotype,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Cells_Location_Center_X,...,Nuclei_Texture_InverseDifferenceMoment_RFP_3_01_256,Nuclei_Texture_InverseDifferenceMoment_RFP_3_02_256,Nuclei_Texture_InverseDifferenceMoment_RFP_3_03_256,Nuclei_Texture_SumEntropy_DAPI_3_02_256,Nuclei_Texture_SumEntropy_RFP_3_01_256,Nuclei_Texture_SumVariance_CY5_3_03_256,Nuclei_Texture_SumVariance_DAPI_3_03_256,Nuclei_Texture_SumVariance_GFP_3_03_256,Nuclei_Texture_SumVariance_RFP_3_01_256,Metadata_Number_of_Cells_Neighbors_Adjacent
0,B,1,B1,10,79,NF1,WT,870.435339,133.774194,863.193505,...,-0.674383,-1.069853,-0.904776,-0.359152,0.951915,-0.562028,-0.550270,-0.536587,0.236774,3.0
1,B,1,B1,11,79,NF1,WT,827.549320,342.283025,810.793536,...,-1.133170,-1.194594,-0.828107,-0.978650,0.081101,-0.698954,-0.702868,-0.473571,-0.310403,0.0
2,B,1,B1,11,79,NF1,WT,427.937346,356.977306,406.334199,...,-0.127992,-0.504128,-0.202547,0.953802,-0.308874,0.079813,0.325203,0.480684,-0.369155,3.0
3,B,1,B1,11,79,NF1,WT,272.036245,389.802436,282.897144,...,0.481350,-0.392029,-0.471200,0.536328,0.350581,-0.434787,-0.169241,-0.097767,-0.261039,2.0
4,B,1,B1,11,79,NF1,WT,944.416824,736.917498,963.654663,...,0.492640,0.033404,0.850012,0.263532,-1.010511,-0.093067,-0.124964,0.162283,-0.455981,0.0


## Load in feature importance data and determine the top two differential features for Null and WT

Top positive feature means the most important in predicting the WT genotype, most negative is most important in predicting Null genotype.

In [ ]:
feat_import_df = pd.read_parquet(
    pathlib.Path(
        "../../2.evaluate_model/model_evaluation_data/feature_importances.parquet"
    )
)

# Find the top positive feature (predicting WT)
correlation_feature = feat_import_df.sort_values(
    by="feature_importances", ascending=False
).iloc[0]["feature_names"]

# Find the top negative feature (predicting Null)
radial_feature = feat_import_df.loc[
    feat_import_df["feature_importances"].idxmin(), "feature_names"
]

# Find the second top negative feature (extra)
intensity_feature = feat_import_df.sort_values(
    by="feature_importances", ascending=True
).iloc[1]["feature_names"]

# Print the features
print(correlation_feature)
print(radial_feature)
print(intensity_feature)

Cells_Correlation_Correlation_DAPI_GFP
Cytoplasm_RadialDistribution_FracAtD_RFP_4of4
Cytoplasm_Intensity_MeanIntensityEdge_GFP


## Filter plate 5 single-cells to only include isolated cells that are not near the edge of the FOV

In [7]:
# Filter the DataFrame directly
filtered_plate5_df = plate5_df[
    (plate5_df["Metadata_Number_of_Cells_Neighbors_Adjacent"].isin([0]))
    & (plate5_df["Metadata_Nuclei_Location_Center_X"] > crop_size // 2)
    & (
        plate5_df["Metadata_Nuclei_Location_Center_X"]
        < (plate5_df["Metadata_Nuclei_Location_Center_X"].max() - crop_size // 2)
    )
    & (plate5_df["Metadata_Nuclei_Location_Center_Y"] > crop_size // 2)
    & (
        plate5_df["Metadata_Nuclei_Location_Center_Y"]
        < (plate5_df["Metadata_Nuclei_Location_Center_Y"].max() - crop_size // 2)
    )
]

print(filtered_plate5_df.shape)
filtered_plate5_df.head()

(589, 1175)


,Metadata_WellRow,Metadata_WellCol,Metadata_Well,Metadata_Site,Metadata_number_of_singlecells,Metadata_gene_name,Metadata_genotype,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Cells_Location_Center_X,...,Nuclei_Texture_InverseDifferenceMoment_RFP_3_01_256,Nuclei_Texture_InverseDifferenceMoment_RFP_3_02_256,Nuclei_Texture_InverseDifferenceMoment_RFP_3_03_256,Nuclei_Texture_SumEntropy_DAPI_3_02_256,Nuclei_Texture_SumEntropy_RFP_3_01_256,Nuclei_Texture_SumVariance_CY5_3_03_256,Nuclei_Texture_SumVariance_DAPI_3_03_256,Nuclei_Texture_SumVariance_GFP_3_03_256,Nuclei_Texture_SumVariance_RFP_3_01_256,Metadata_Number_of_Cells_Neighbors_Adjacent
1,B,1,B1,11,79,NF1,WT,827.549320,342.283025,810.793536,...,-1.133170,-1.194594,-0.828107,-0.978650,0.081101,-0.698954,-0.702868,-0.473571,-0.310403,0.0
4,B,1,B1,11,79,NF1,WT,944.416824,736.917498,963.654663,...,0.492640,0.033404,0.850012,0.263532,-1.010511,-0.093067,-0.124964,0.162283,-0.455981,0.0
16,B,1,B1,15,79,NF1,WT,730.918660,125.325997,651.000090,...,-1.270665,-1.532210,-1.337768,0.327361,0.669852,-0.648986,-0.233956,-0.470126,-0.181658,0.0
17,B,1,B1,15,79,NF1,WT,242.052322,187.476471,228.928538,...,0.560580,0.042825,0.432985,0.876738,-0.020096,0.363864,0.180029,1.173320,-0.329673,0.0
21,B,1,B1,17,79,NF1,WT,1059.657581,254.698862,1009.748789,...,2.412206,1.141638,1.203298,-0.364632,-0.520605,-0.601560,-0.562214,-0.469391,-0.385051,0.0


### Max single-cells for Correlation feature (represent WT)

In [8]:
# Get data frame with the next top 6 single-cells from the top WT coefficient
max_corr_feature = filtered_plate5_df[
    filtered_plate5_df["Metadata_genotype"] == "WT"
].nlargest(18, correlation_feature).iloc[12:18][
    [
        correlation_feature,
        "Metadata_genotype",
        "Metadata_Well",
        "Metadata_Plate",
        "Metadata_Site",
        "Metadata_Number_of_Cells_Neighbors_Adjacent",
        "Metadata_Nuclei_Location_Center_X",
        "Metadata_Nuclei_Location_Center_Y",
    ]
]

# Append the DataFrame and its name to the lists
list_of_dfs.append(max_corr_feature)
list_of_names.append("max_corr_feature")

print(max_corr_feature.shape)
max_corr_feature

(6, 8)


,Cells_Correlation_Correlation_DAPI_GFP,Metadata_genotype,Metadata_Well,Metadata_Plate,Metadata_Site,Metadata_Number_of_Cells_Neighbors_Adjacent,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y
2259,2.154344,WT,F4,Plate_5,20,0.0,953.548347,266.444167
858,2.103369,WT,E2,Plate_5,3,0.0,580.649195,491.926437
2071,2.092754,WT,D4,Plate_5,7,0.0,847.818446,609.990922
2133,1.963435,WT,E4,Plate_5,14,0.0,382.599875,682.793394
4,1.876981,WT,B1,Plate_5,11,0.0,944.416824,736.917498
2310,1.871065,WT,G4,Plate_5,15,0.0,213.953227,551.401777


### Min single-cells for Correlation feature (represent Null)

In [9]:
# Get data frame with the top 3 single-cells from the top WT coefficient
min_corr_feature = filtered_plate5_df[
    filtered_plate5_df["Metadata_genotype"] == "Null"
].nsmallest(6, correlation_feature)[
    [
        correlation_feature,
        "Metadata_genotype",
        "Metadata_Well",
        "Metadata_Plate",
        "Metadata_Site",
        "Metadata_Number_of_Cells_Neighbors_Adjacent",
        "Metadata_Nuclei_Location_Center_X",
        "Metadata_Nuclei_Location_Center_Y",
    ]
]

# Append the DataFrame and its name to the lists
list_of_dfs.append(min_corr_feature)
list_of_names.append("min_corr_feature")

print(min_corr_feature.shape)
min_corr_feature

(6, 8)


,Cells_Correlation_Correlation_DAPI_GFP,Metadata_genotype,Metadata_Well,Metadata_Plate,Metadata_Site,Metadata_Number_of_Cells_Neighbors_Adjacent,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y
2438,-4.755442,Null,B9,Plate_5,19,0.0,431.907657,629.735016
2841,-2.671526,Null,D9,Plate_5,17,0.0,691.091827,728.043960
2735,-2.214581,Null,D9,Plate_5,10,0.0,355.892197,565.846509
3873,-1.951390,Null,F10,Plate_5,14,0.0,761.724484,481.483342
3733,-1.903057,Null,E10,Plate_5,14,0.0,1034.177966,163.005932
3798,-1.619453,Null,E10,Plate_5,5,0.0,1017.167832,334.194493


### Max single-cells for Intensity feature (represent Null)

In [10]:
# Get data frame with the top 3 single-cells from the second top Null coefficient
max_int_feature = filtered_plate5_df[
    filtered_plate5_df["Metadata_genotype"] == "Null"
].nlargest(6, intensity_feature)[
    [
        intensity_feature,
        "Metadata_genotype",
        "Metadata_Well",
        "Metadata_Plate",
        "Metadata_Site",
        "Metadata_Number_of_Cells_Neighbors_Adjacent",
        "Metadata_Nuclei_Location_Center_X",
        "Metadata_Nuclei_Location_Center_Y",
    ]
]

# Append the DataFrame and its name to the lists
list_of_dfs.append(max_int_feature)
list_of_names.append("max_int_feature")

print(max_int_feature.shape)
max_int_feature

(6, 8)


,Cytoplasm_Intensity_MeanIntensityEdge_GFP,Metadata_genotype,Metadata_Well,Metadata_Plate,Metadata_Site,Metadata_Number_of_Cells_Neighbors_Adjacent,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y
4303,5.036352,Null,C11,Plate_5,16,0.0,939.361512,412.432302
3509,4.876263,Null,C10,Plate_5,4,0.0,951.492215,667.124058
3442,4.647287,Null,C10,Plate_5,11,0.0,645.682606,256.935204
2438,4.221244,Null,B9,Plate_5,19,0.0,431.907657,629.735016
4613,4.186915,Null,E11,Plate_5,10,0.0,682.757504,396.595056
4320,3.544175,Null,C11,Plate_5,18,0.0,701.753709,618.162018


### Min single-cells for Intensity feature (represent WT)

In [11]:
# Get data frame with the top 3 single-cells from the second top Null coefficient
min_int_feature = filtered_plate5_df[
    filtered_plate5_df["Metadata_genotype"] == "WT"
].nsmallest(6, intensity_feature)[
    [
        intensity_feature,
        "Metadata_genotype",
        "Metadata_Well",
        "Metadata_Plate",
        "Metadata_Site",
        "Metadata_Number_of_Cells_Neighbors_Adjacent",
        "Metadata_Nuclei_Location_Center_X",
        "Metadata_Nuclei_Location_Center_Y",
    ]
]

# Append the DataFrame and its name to the lists
list_of_dfs.append(min_int_feature)
list_of_names.append("min_int_feature")

print(min_int_feature.shape)
min_int_feature

(6, 8)


,Cytoplasm_Intensity_MeanIntensityEdge_GFP,Metadata_genotype,Metadata_Well,Metadata_Plate,Metadata_Site,Metadata_Number_of_Cells_Neighbors_Adjacent,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y
2296,-1.316602,WT,F4,Plate_5,8,0.0,434.582913,513.323720
994,-1.296221,WT,F2,Plate_5,5,0.0,474.847136,689.379705
846,-1.292468,WT,E2,Plate_5,21,0.0,813.353410,481.614144
2207,-1.290977,WT,E4,Plate_5,18,0.0,367.469324,544.971473
989,-1.197940,WT,F2,Plate_5,19,0.0,977.606305,159.600154
1736,-1.163598,WT,G3,Plate_5,21,0.0,369.324843,532.509997


### Max single-cells for Radial Distribution feature (represent Null)

In [12]:
# Get data frame with the top 3 single-cells from the top Null coefficient
max_radial_feature = filtered_plate5_df[
    filtered_plate5_df["Metadata_genotype"] == "Null"
].nlargest(6, radial_feature)[
    [
        radial_feature,
        "Metadata_genotype",
        "Metadata_Well",
        "Metadata_Plate",
        "Metadata_Site",
        "Metadata_Number_of_Cells_Neighbors_Adjacent",
        "Metadata_Nuclei_Location_Center_X",
        "Metadata_Nuclei_Location_Center_Y",
    ]
]

# Append the DataFrame and its name to the lists
list_of_dfs.append(max_radial_feature)
list_of_names.append("max_radial_feature")

print(max_radial_feature.shape)
max_radial_feature

(6, 8)


,Cytoplasm_RadialDistribution_FracAtD_RFP_4of4,Metadata_genotype,Metadata_Well,Metadata_Plate,Metadata_Site,Metadata_Number_of_Cells_Neighbors_Adjacent,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y
4779,3.002824,Null,E11,Plate_5,5,0.0,278.493498,255.725618
4749,2.321686,Null,E11,Plate_5,7,0.0,936.565267,639.347328
2792,2.125977,Null,D9,Plate_5,7,0.0,143.412133,499.391430
3374,2.069727,Null,B10,Plate_5,17,0.0,1035.746981,574.287047
4711,1.928103,Null,E11,Plate_5,8,0.0,823.467270,144.731894
4370,1.877431,Null,C11,Plate_5,19,0.0,337.705552,732.650916


### Min single-cells for Radial Distribution feature (represent WT)

In [13]:
# Get data frame with the top 3 single-cells from the top Null coefficient
min_radial_feature = filtered_plate5_df[
    filtered_plate5_df["Metadata_genotype"] == "WT"
].nsmallest(6, radial_feature)[
    [
        radial_feature,
        "Metadata_genotype",
        "Metadata_Well",
        "Metadata_Plate",
        "Metadata_Site",
        "Metadata_Number_of_Cells_Neighbors_Adjacent",
        "Metadata_Nuclei_Location_Center_X",
        "Metadata_Nuclei_Location_Center_Y",
    ]
]

# Append the DataFrame and its name to the lists
list_of_dfs.append(min_radial_feature)
list_of_names.append("min_radial_feature")

print(min_radial_feature.shape)
min_radial_feature

(6, 8)


,Cytoplasm_RadialDistribution_FracAtD_RFP_4of4,Metadata_genotype,Metadata_Well,Metadata_Plate,Metadata_Site,Metadata_Number_of_Cells_Neighbors_Adjacent,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y
1238,-2.880913,WT,C3,Plate_5,11,0.0,1064.825597,676.707158
176,-2.445647,WT,D1,Plate_5,5,0.0,223.524429,485.600628
236,-2.425442,WT,E1,Plate_5,13,0.0,543.470981,161.104384
722,-2.310212,WT,D2,Plate_5,10,0.0,166.289499,681.268685
2177,-2.277879,WT,E4,Plate_5,7,0.0,610.201000,349.409732
563,-2.247404,WT,B2,Plate_5,11,0.0,452.365788,706.733810


## Merge feature info into dictionary for processing

In [14]:
sc_dict = create_sc_dict(dfs=list_of_dfs, names=list_of_names)

# Check the created dictionary for the first two items
pprint(list(sc_dict.items())[:2], indent=4)

[   (   'max_corr_feature_1',
        {   'location_center_x': 953.5483468496569,
            'location_center_y': 266.44416718652525,
            'plate': 'Plate_5',
            'site': '20',
            'well': 'F4'}),
    (   'max_corr_feature_2',
        {   'location_center_x': 580.6491954022988,
            'location_center_y': 491.9264367816092,
            'plate': 'Plate_5',
            'site': '3',
            'well': 'E2'})]


## Generate single-cell crops 

In [15]:
generate_sc_crops(
    sc_dict=sc_dict,
    channel_mapping=channel_mapping,
    images_dir=images_dir,
    output_img_dir=output_img_dir,
    crop_size=crop_size,
)

File sc_crops/min_corr_feature_1/min_corr_feature_1_d1_cropped.png already exists. Skipping.
File sc_crops/min_corr_feature_1/min_corr_feature_1_d2_cropped.png already exists. Skipping.
File sc_crops/min_corr_feature_1/min_corr_feature_1_d3_cropped.png already exists. Skipping.
File sc_crops/min_corr_feature_1/min_corr_feature_1_d4_cropped.png already exists. Skipping.
File sc_crops/min_corr_feature_2/min_corr_feature_2_d1_cropped.png already exists. Skipping.
File sc_crops/min_corr_feature_2/min_corr_feature_2_d2_cropped.png already exists. Skipping.
File sc_crops/min_corr_feature_2/min_corr_feature_2_d3_cropped.png already exists. Skipping.
File sc_crops/min_corr_feature_2/min_corr_feature_2_d4_cropped.png already exists. Skipping.
File sc_crops/min_corr_feature_3/min_corr_feature_3_d1_cropped.png already exists. Skipping.
File sc_crops/min_corr_feature_3/min_corr_feature_3_d2_cropped.png already exists. Skipping.
File sc_crops/min_corr_feature_3/min_corr_feature_3_d3_cropped.png alr